<a href="https://colab.research.google.com/github/desaiankitb/video-capabilities/blob/main/Copy_of_CoDeF_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content
%env TF_CPP_MIN_LOG_LEVEL=1

# !git clone --recursive https://github.com/nvlabs/tiny-cuda-nn
# %cd /content/tiny-cuda-nn
# !cmake . -B build
# !cmake --build build --config RelWithDebInfo -j
# !pip install git+https://github.com/NVlabs/tiny-cuda-nn/#subdirectory=bindings/torch

!git clone -b dev https://github.com/camenduru/CoDeF
%cd /content/CoDeF

!pip install -q pytorch_lightning einops ipdb torch_optimizer kornia scikit-video
!pip install -q https://huggingface.co/camenduru/CoDeF/resolve/main/tinycudann-1.7-cp310-cp310-linux_x86_64.whl

!apt -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/CoDeF/resolve/main/all_sequences.zip -d /content/CoDeF/ckpts -o all_sequences.zip
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/CoDeF/resolve/main/beauty_0.ckpt -d /content/CoDeF/ckpts -o beauty_0.ckpt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/CoDeF/resolve/main/beauty_1.ckpt -d /content/CoDeF/ckpts -o beauty_1.ckpt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/CoDeF/resolve/main/lemon_hit.ckpt -d /content/CoDeF/ckpts -o lemon_hit.ckpt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/CoDeF/resolve/main/scene_0.ckpt -d /content/CoDeF/ckpts -o scene_0.ckpt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/CoDeF/resolve/main/white_smoke.ckpt -d /content/CoDeF/ckpts -o white_smoke.ckpt

!unzip /content/CoDeF/ckpts/all_sequences.zip -d /content/CoDeF/ckpts
!rm -rf /content/CoDeF/ckpts/__MACOSX

# !mkdir /content/CoDeF/ckpts/all_sequences/scene_0/base
# !cp /content/CoDeF/ckpts/scene_0.ckpt /content/CoDeF/ckpts/all_sequences/scene_0/base/scene_0.ckpt

/content
env: TF_CPP_MIN_LOG_LEVEL=1
Cloning into 'CoDeF'...
remote: Enumerating objects: 146, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 146 (delta 15), reused 14 (delta 14), pack-reused 127
Receiving objects: 100% (146/146), 54.29 MiB | 28.33 MiB/s, done.
Resolving deltas: 100% (45/45), done.
/content/CoDeF
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.7/727.7 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.7/705.7 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB

In [2]:
# @title Train (Step 1)

GPUS=0
NAME="beauty_0"
EXP_NAME="base"
ROOT_DIRECTORY=f"ckpts/all_sequences/{NAME}/{NAME}"
MODEL_SAVE_PATH=f"ckpts/all_sequences/{NAME}"
LOG_SAVE_PATH=f"logs/test_all_sequences/{NAME}"
WEIGHT_PATH=f"ckpts/all_sequences/{NAME}.ckpt"
CONFIG_DIRECTORY=f"configs/{NAME}/{EXP_NAME}.yaml"
MASK_DIRECTORY=f"all_sequences/{NAME}/{NAME}_masks_0 all_sequences/{NAME}/{NAME}_masks_1"
FLOW_DIRECTORY=f"all_sequences/{NAME}/{NAME}_flow"

!python train.py --root_dir $ROOT_DIRECTORY \
                --model_save_path $MODEL_SAVE_PATH \
                --log_save_path $LOG_SAVE_PATH  \
                --gpus $GPUS \
                --encode_w --annealed \
                --config $CONFIG_DIRECTORY \
                --exp_name $EXP_NAME
                # --mask_dir $MASK_DIRECTORY
                # --flow_dir $FLOW_DIRECTORY

ModelCheckpoint(save_last=True, save_top_k=-1, monitor=None) will duplicate the last checkpoint saved.
/usr/local/lib/python3.10/dist-packages/lightning_fabric/connector.py:554: UserWarning: 16 is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
  rank_zero_warn(
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/1
----------------------------------------------------------------------------------------------------
distributed_backend=nccl
All distributed processes registered. Starting with 1 processes
----------------------------------------------------------------------------------------------------

Missing logger folder: logs/test_all_sequences/beauty_0/base
2023-10-05 10:24:32.020953: W tensorflow/compiler/tf2tensorrt/utils/py_u

In [3]:
# GPUS=0
# NAME="beauty_0"
# EXP_NAME="base"
# ROOT_DIRECTORY=f"ckpts/all_sequences/{NAME}/{NAME}"
# MODEL_SAVE_PATH=f"ckpts/all_sequences/{NAME}"
# LOG_SAVE_PATH=f"logs/test_all_sequences/{NAME}"
# WEIGHT_PATH=f"ckpts/all_sequences/{NAME}.ckpt"
# CONFIG_DIRECTORY=f"configs/{NAME}/{EXP_NAME}.yaml"
# MASK_DIRECTORY=f"all_sequences/{NAME}/{NAME}_masks_0 all_sequences/{NAME}/{NAME}_masks_1"
# FLOW_DIRECTORY=f"all_sequences/{NAME}/{NAME}_flow"

# @title Generate Canonical Image (Step 2)

GPUS=0
NAME="beauty_0"
EXP_NAME="base"
ROOT_DIRECTORY=f"ckpts/all_sequences/{NAME}/{NAME}"
LOG_SAVE_PATH=f"logs/test_all_sequences/{NAME}"
MASK_DIRECTORY=f"all_sequences/{NAME}/{NAME}_masks_0 all_sequences/{NAME}/{NAME}_masks_1"
WEIGHT_PATH=f"ckpts/all_sequences/{NAME}/{EXP_NAME}/last.ckpt"
CONFIG_DIRECTORY=f"configs/{NAME}/{EXP_NAME}.yaml"

!python train.py  --test --encode_w \
                  --root_dir $ROOT_DIRECTORY \
                  --log_save_path $LOG_SAVE_PATH \
                  --mask_dir $MASK_DIRECTORY \
                  --weight_path $WEIGHT_PATH \
                  --gpus $GPUS \
                  --config $CONFIG_DIRECTORY \
                  --exp_name $EXP_NAME \
                  --save_deform False

ModelCheckpoint(save_last=True, save_top_k=-1, monitor=None) will duplicate the last checkpoint saved.
/usr/local/lib/python3.10/dist-packages/lightning_fabric/connector.py:554: UserWarning: 16 is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
  rank_zero_warn(
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
ckpts/all_sequences/beauty_0/beauty_0/00000.png
ckpts/all_sequences/beauty_0/beauty_0/00001.png
ckpts/all_sequences/beauty_0/beauty_0/00002.png
ckpts/all_sequences/beauty_0/beauty_0/00003.png
ckpts/all_sequences/beauty_0/beauty_0/00004.png
ckpts/all_sequences/beauty_0/beauty_0/00005.png
ckpts/all_sequences/beauty_0/beauty_0/00006.png
ckpts/all_sequences/beauty_0/beauty_0/00007.png
ckpts/all_sequences/beauty_0/beauty_0/00008.png
ckpts/all_sequences/beauty_0/beauty_0/00009.png

In [12]:
# @title Generate Video (Step 3)

GPUS=0
NAME="beauty_0"
EXP_NAME="base"
ROOT_DIRECTORY=f"ckpts/all_sequences/{NAME}/{NAME}"
ROOT_DIRECTORY=f"/content/CoDeF/ckpts/all_sequences/beauty_0/beauty_0"
LOG_SAVE_PATH=f"logs/test_all_sequences/{NAME}"
MASK_DIRECTORY=f"all_sequences/{NAME}/{NAME}_masks_0 all_sequences/{NAME}/{NAME}_masks_1"
CONFIG_DIRECTORY=f"configs/{NAME}/{EXP_NAME}.yaml"
CANONICAL_DIR=f"ckpts/all_sequences/{NAME}/{EXP_NAME}_control"
CANONICAL_DIR=f'/content/CoDeF/ckpts/all_sequences/beauty_0/beauty_0'
WEIGHT_PATH=f"ckpts/all_sequences/{NAME}/{EXP_NAME}/last.ckpt"
# WEIGHT_PATH=f"ckpts/all_sequences/{NAME}.ckpt"

!python train.py --test --encode_w \
                --root_dir $ROOT_DIRECTORY \
                --log_save_path $LOG_SAVE_PATH \
                --mask_dir $MASK_DIRECTORY \
                --weight_path $WEIGHT_PATH \
                --gpus $GPUS \
                --canonical_dir $CANONICAL_DIR \
                --config $CONFIG_DIRECTORY \
                --exp_name $EXP_NAME

ModelCheckpoint(save_last=True, save_top_k=-1, monitor=None) will duplicate the last checkpoint saved.
/usr/local/lib/python3.10/dist-packages/lightning_fabric/connector.py:554: UserWarning: 16 is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
  rank_zero_warn(
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/content/CoDeF/ckpts/all_sequences/beauty_0/beauty_0/00000.png
/content/CoDeF/ckpts/all_sequences/beauty_0/beauty_0/00001.png
/content/CoDeF/ckpts/all_sequences/beauty_0/beauty_0/00002.png
/content/CoDeF/ckpts/all_sequences/beauty_0/beauty_0/00003.png
/content/CoDeF/ckpts/all_sequences/beauty_0/beauty_0/00004.png
/content/CoDeF/ckpts/all_sequences/beauty_0/beauty_0/00005.png
/content/CoDeF/ckpts/all_sequences/beauty_0/beauty_0/00006.png
/content/CoDeF/ckpts/all_sequences/bea